# SQL Optimizer Demo

This single-cell demo analyzes a pasted SQL query, suggests indexes, provides heuristic rewrites, and optionally runs `EXPLAIN QUERY PLAN` if you upload CSV/DDL files. Run each cell in Colab.

In [ ]:
# One-cell SQL Optimizer Demo (Colab)
# Paste this cell into Google Colab and run. Upload CSV/DDL files if you want EXPLAIN support.

try:
    # Colab / Notebook installs
    import sys
    !pip -q install sqlparse sql_metadata sqlglot openai pandas
except Exception:
    pass

import sqlparse, os, re, json
from sql_metadata import Parser
import sqlglot
from google.colab import files
import sqlite3
import pandas as pd

def print_h1(s):
    print('\n' + '='*len(s))
    print(s)
    print('='*len(s) + '\n')

print_h1('STEP 1 — Paste SQL (end with empty line)')
print('Paste your SQL (multi-line). Press Enter twice to finish:')
lines = []
while True:
    try:
        l = input()
    except EOFError:
        break
    if l.strip()=='' and len(lines)>0:
        break
    lines.append(l)
user_sql = '\n'.join(lines).strip()
if not user_sql:
    print('No SQL provided — exiting.')
    raise SystemExit

print_h1('Formatted SQL')
print(sqlparse.format(user_sql, reindent=True, keyword_case='upper'))

print_h1('Parsing query structure')
try:
    parser = Parser(user_sql)
    tables = parser.tables
    columns = parser.columns
    where_columns = parser.columns_dict.get('where', [])
except Exception as e:
    print('Parsing error:', e)
    tables, columns, where_columns = [], [], []

print('Tables detected:', tables)
print('Columns detected (sample):', columns[:50])
print('WHERE columns:', where_columns)

print_h1('Heuristic suggestions & index recommendations')
suggestions = []
suggestions.append('Avoid SELECT * in production; list required columns to reduce IO.')
suggestions.append('Use LIMIT for exploratory queries to avoid large result sets.')
suggestions.append('Prefer explicit JOIN types and ensure join/filter columns are indexed.')

# Build index recommendations heuristically
index_recommendations = []
filter_cols = set(where_columns)
on_matches = re.findall(r"ON\s+([^\n\r]+?)(?:\s+WHERE|\s+JOIN|\)|$)", user_sql, flags=re.I)
on_cols = []
for m in on_matches:
    on_cols += re.findall(r"([A-Za-z_][A-Za-z0-9_]*\.[A-Za-z_][A-Za-z0-9_]*)", m)
    on_cols += re.findall(r"([A-Za-z_][A-Za-z0-9_]*)\s*=\s*[A-Za-z_][A-Za-z0-9_]*", m)
filter_cols.update([c.split('.')[-1] for c in on_cols if '.' in c] + on_cols)

groupby_cols = []
m = re.search(r"GROUP\s+BY\s+([^\n\r]+?)(?:\s+ORDER|\s+HAVING|\)|$)", user_sql, flags=re.I)
if m:
    gb = re.split(r",\s*", m.group(1).strip())
    groupby_cols = [re.sub(r"\W", "", g) for g in gb]

for t in tables:
    rec_cols = []
    pref = []
    for c in filter_cols:
        if '.' in c and c.split('.')[0].lower() == t.lower():
            pref.append(c.split('.')[-1])
    rec_cols += pref
    common_cols = ["id","user_id","created_at","updated_at"]
    for cc in common_cols:
        if cc in columns and len(rec_cols)<3:
            rec_cols.append(cc)
    for g in groupby_cols:
        if len(rec_cols)<3:
            rec_cols.append(g)
    for c in filter_cols:
        if len(rec_cols)<3 and c in columns:
            rec_cols.append(c)
    if rec_cols:
        index_recommendations.append({'table':t, 'columns':rec_cols[:3]})
        suggestions.append(f"Consider indexes on {t}({', '.join(rec_cols[:3])}) to speed WHERE and JOIN filters.")

for s in suggestions:
    print('-', s)

print_h1('Generated CREATE INDEX statements (heuristic)')
for rec in index_recommendations:
    cols = rec['columns']
    stmt = f"CREATE INDEX idx_{rec['table']}_{'_'.join(cols)} ON {rec['table']}({', '.join(cols)});"
    print(stmt)

print_h1('Attempted rewrites (simple heuristics)')
rewrites = []
sel = re.search(r"SELECT\s+(.*?)\s+FROM", user_sql, flags=re.I|re.S)
selected_cols = []
if sel:
    sel_cols = re.split(r",\s*", sel.group(1).strip())
    selected_cols = [re.sub(r".*\.","", re.sub(r"\W", "", c)) for c in sel_cols if c.strip() != "*"]
if '*' in user_sql and selected_cols:
    explicit = ', '.join(selected_cols)
    rewrites.append(('Replace SELECT * with explicit columns', re.sub(r"SELECT\s+\*", f"SELECT {explicit}", user_sql, flags=re.I)))
if rewrites:
    for title, q in rewrites:
        print('->', title)
        print(sqlparse.format(q, reindent=True, keyword_case='upper'))
else:
    print('No safe heuristic rewrites generated.')

print_h1('Optional: Upload CSV/DDL files to run EXPLAIN (press Enter to skip)')
print('Upload now (use the file upload dialog) or press Enter to skip:')
up = files.upload()
if up:
    conn = sqlite3.connect('temp_db.sqlite')
    for fname in up.keys():
        print('Processing', fname)
        if fname.lower().endswith('.csv'):
            tname = os.path.splitext(os.path.basename(fname))[0]
            df_csv = pd.read_csv(fname)
            df_csv.to_sql(tname, conn, if_exists='replace', index=False)
            print(f'Loaded {tname} rows={len(df_csv)}')
        elif fname.lower().endswith('.sql'):
            ddl = open(fname).read()
            try:
                conn.executescript(ddl)
                print('Executed DDL')
            except Exception as e:
                print('DDL error', e)
    try:
        cur = conn.cursor()
        cur.execute(f'EXPLAIN QUERY PLAN {user_sql}')
        rows = cur.fetchall()
        print('\nSQLite EXPLAIN QUERY PLAN:')
        for r in rows:
            print(r)
    except Exception as e:
        print('EXPLAIN failed:', e)

print_h1('Final Report (saved)')
report = {
    'original_sql': user_sql,
    'tables': tables,
    'columns': columns[:200],
    'suggestions': suggestions,
    'index_recommendations': index_recommendations,
    'rewrites': [q for _,q in rewrites]
}
plain = []
plain.append('SQL Query Optimizer Report')
plain.append('-'*40)
plain.append('Original SQL:')
plain.append(user_sql)
plain.append('\nDetected tables: ' + ', '.join(tables))
plain.append('\nSuggestions:')
for s in suggestions:
    plain.append('- ' + s)
plain.append('\nIndex recommendations:')
for rec in index_recommendations:
    cols = rec['columns']
    plain.append(f"- {rec['table']}({', '.join(cols)})")
plain_text = '\n'.join(plain)
with open('sql_optimizer_report.txt','w') as f:
    f.write(plain_text)
print('Saved report to sql_optimizer_report.txt')
files.download('sql_optimizer_report.txt')
if index_recommendations:
    with open('suggested_indexes.sql','w') as f:
        for rec in index_recommendations:
            cols = rec['columns']
            f.write(f"CREATE INDEX idx_{rec['table']}_{'_'.join(cols)} ON {rec['table']}({', '.join(cols)});\n")
    print('Saved suggested_indexes.sql')

print('\nDone. Use the generated suggestions in a staging environment before applying in production.')
